In [ ]:
# pip install pymupdf4llm sentence-transformers chromadb ollama langchain
import pymupdf4llm
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PDF 파일 경로
file_path = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"
# 로컬 인공지능 언어 모델 
model = "exaone3.5:latest"

def load_pdf(file_path):
    pdf_data = pymupdf4llm.to_markdown(file_path)
    text = "".join(pdf_data)  # 페이지별 텍스트를 하나의 문자열로 결합
    print("PDF파일 로드..")
    return text

def split_text(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
    return text_splitter.split_text(text)

In [2]:
from sentence_transformers import SentenceTransformer
import chromadb

embedder = SentenceTransformer("intfloat/multilingual-e5-small")
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="rag_collection",  metadata={"hnsw:space": "cosine"} )

print("문서 로드 및 임베딩 시작...")

raw_text = load_pdf(file_path)
chunks = split_text(raw_text)
embeddings = embedder.encode(chunks, convert_to_tensor=False)

for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    collection.add(
        ids=[f"chunk_{i + 1}"],
        embeddings=[embedding.tolist()],
        metadatas=[{"text": chunk}],
    )

/Users/a1000074/dev/PyProjects/langchain_study2/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name intfloat/multilingual-e5-small. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /repos/61/17/6117d6a86f40a6a2824a30a33ac957b0b55f3850beda3c389e4e8c2a1f9562d3/1a55775f53449dac10a2bcbc312469fac40b96d53198c407081a831f81c98477?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1750147744&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDE0Nzc0NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82MS8xNy82MTE3ZDZhODZmNDBhNmEyODI0YTMwYTMzYWM5NTdiMGI1NWYzODUwYmVkYTNjMzg5ZTRlOGMyYTFmOTU2MmQzLzFhNTU3NzVmNTM0NDlkYWMxMGEyYmNiYzMxMjQ2OWZhYzQwYjk2ZDUzMTk4YzQwNzA4MWE4MzFmODFjOTg0Nzc~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=CnHMLBerm02ZuNbMa8sTdaNAEA4k4v67CnecZCey~Jw5ksVQlwxaB7kMU8o6anUNGJhSicKRCqPSfYAyfJdgS-yuhisa8NliiWuF4VCu35svaXEYgw2X0a16lo9-XdG4yw9yhO7QFPlSHjqN8L-rADegG4Bnktt37IJDbyPR4zKucYJPTn8RjvZnHdecHew7n72NgY~qiQavR1pCj